In [61]:
# Assume geometric Brownian motion
# mu = drift = expected return
# sigma = standard deviation of return (the stochastic piece)
# D = drawdown (10% = 0.10)
%precision %.2f
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

In [69]:
import locale
locale.setlocale(locale.LC_ALL, '')  # Use '' for auto, or force e.g. to 'en_US.UTF-8'

'English_United Kingdom.1252'

In [41]:
def prob_drawdown_pct(D, T, mu, sigma):
    # Laplace transformation method to estimate max drawdown
    # D = drawdown (10% = 0.10)
    # T = period in years
    # mu = drift = expected return (annualised)
    # sigma = annualised standard deviation of return (the stochastic piece)
    # define Γ (gamma) = 2mu / sigma^2
    gamma = (2 * mu * T) / ((sigma * T) ** 2)
    # probability p of a drawdown greater than D is approximately: p = [1 / (1 – D)]^(-Γ))
    return (1 / (1 - D))**(-gamma)

## Translate VaR to Annual PnL target

In [71]:
# VaR @ 95% is 1.645 s.d.
# assume $100k VaR.... back out implied portfolio size:
#
# first step: get daily standard deviation from 95% VaR (which is 1.645 standard deviations)
sd = 100000 / 1.645
print(f'{sd:n}')

60,790.3


In [72]:
# Daily VaR / Standard Deviation, is proportional to annual standard deviation
# proportion is sqrt(Time), so 252 trading days per year means
# daily standard deviation is annual s.d. / sqrt(252)

# take previously calculated s.d. and gross up for time

sd_annual = sd * np.sqrt(252)
print(f'{sd_annual:n}')

965,016


In [73]:
# Assume a realistic attainable Sharpe Ratio of 0.75
# This is annualised return / annualised volatility of that return
#
# exptected annual return of a portfolio = sharpe ratio * annualised volatility

expected_return = 0.75 * sd_annual
print(f'{expected_return:n}')


723,762


In [ ]:
# so with a VaR of $100k, a strategy with Sharpe = 0.75 can be expected to make $723k
# hence daily VaR to PnL target is approximately 10x for 1:1 unit of return:risk 
# (this is because the annualising factor sqrt(252) is approx 16 and
# a 95% daily VaR number uses 1.64 standard deviations
# 15.87 / 1.645 = 9.65 = approx 10)

# so for a 0.75 Sharpe Ratio portfolio, er expect 0.75 * 9.65 = 7.2x

## Translate VaR into monthly drawdown

In [75]:
# Daily (1 day) --> Monthly (22 days)
# proportional by sqrt(22)
# If 1 day 95% VaR is $100k
# then 1 month 95% VaR is
var_95_daily = 100000
var_95_monthly = var_95_daily * np.sqrt(22)
print(f'{var_95_monthly:n}')

469,042


In [77]:
# This implies, 5% of the time we should expect a loss of more than 469k 
# First, let's set monthly 95% VaR to 400k
var_95_monthly = 400000

# If we want to increase this percentage, say we are comfortable with a 10% chance of this
# Then, 90% cumulative normal distribution = 1.28 standard deviations from mean
# compared to 95% = 1.645 s.d.'s
# so, to hit -400k 10% of the time (instead of 5% of the time) we can scale by 
# number of s.d's

var_95_monthly_scaled_to_90 = var_95_monthly * 1.645 / 1.28
print(f'{var_95_monthly_scaled_to_90:n}')

514,062


In [79]:
# scale this number back to daily, to get the imputed daily 95% VaR we need
# that equates to the 10% chance of a 400K monthly drawdown

target_var = var_95_monthly_scaled_to_90 / np.sqrt(22)
print(f'{target_var:n}')

109,598
